# The Vanishing/Exploding Gradients Problems
 * The vanishing gradients problem occurs when the gradients of the loss function with respect to the parameters become extremely small during backpropagation.
 * Conversely, the exploding gradients problem arises when the gradients of the loss function grow exponentially as they propagate backward through the layers during backpropagation.


## Weight Initialization
 * Proper initialization of network weights can help alleviate both vanishing and exploding gradients problems. 
 * Techniques like __Xavier__ initialization and __He__ initialization are commonly used to ensure that the weights are initialized in a way that keeps the signal propagated through the network.

In [2]:
# Importing essential package
import tensorflow as tf
from tensorflow import keras

# List comprehension to get the names of initializers in Keras without underscores
initializer_names = [name for name in dir(keras.initializers) if not name.startswith("_")]

# Printing the initializer names
print(initializer_names)

['Constant', 'GlorotNormal', 'GlorotUniform', 'HeNormal', 'HeUniform', 'Identity', 'Initializer', 'LecunNormal', 'LecunUniform', 'Ones', 'Orthogonal', 'RandomNormal', 'RandomUniform', 'TruncatedNormal', 'VarianceScaling', 'Zeros', 'constant', 'deserialize', 'get', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform', 'identity', 'lecun_normal', 'lecun_uniform', 'ones', 'orthogonal', 'random_normal', 'random_uniform', 'serialize', 'truncated_normal', 'variance_scaling', 'zeros']


* Creating a dense layer with built-in initializers

In [3]:
# Creating a dense layer with 10 units, ReLU activation, and He normal weight initialization
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

* Defining an initializer with Variance Scaling (custom kernel initializer)

In [4]:
# Defining an initializer with Variance Scaling
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')

# Creating a dense layer with 10 units, ReLU activation, and custom kernel initializer
dense_layer = keras.layers.Dense(10, activation="relu", kernel_initializer=init)

## Nonsaturating Activation Functions
 * Non-saturating activation functions refer to activation functions that do not suffer from the vanishing gradient problem to the same extent as traditional saturating activation functions like sigmoid and tanh. These functions typically have gradients that do not diminish as quickly as the __sigmoid__ or __tanh__ functions for large input values, thereby alleviating the vanishing gradient problem. 
 * One prominent example of a non-saturating activation function is the Rectified Linear Unit (__ReLU__) and its variants.

1. __Rectified Linear Unit (ReLU):__
    * ReLU is a popular activation function in deep learning, defined as __f(x)=max(0,x)__. It outputs zero for negative inputs and passes positive inputs unchanged. 
    * It's computationally efficient and doesn't suffer from vanishing gradients for positive inputs. However, it may lead to the __"dying ReLU"__ problem, causing neurons to become inactive for negative inputs. 
    * Variants like __Leaky ReLU__, __Parametric ReLU (PReLU)__, and __Exponential Linear Unit (ELU)__ address this issue by allowing a small, non-zero gradient for negative inputs.

In [5]:
# Listing ReLU variants in Keras.layers module
[m for m in dir(keras.layers) if "relu" in m.lower()]

['LeakyReLU', 'PReLU', 'ReLU', 'ThresholdedReLU']

2. __Leaky ReLU:__
    * Leaky ReLU is an activation function similar to ReLU but with a small, non-zero gradient for negative input values. 
    * It's defined as __f(x)=max(αx,x)__, where α is a small constant (e.g., 0.01). 
    * Leaky ReLU addresses the __"dying ReLU"__ problem by preventing neurons from becoming inactive for negative inputs during training. It retains the efficiency of ReLU while mitigating its limitations.

In [6]:
# Training a neural network on Fashion MNIST using the Leaky ReLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Defining the neural network model with Leaky ReLU activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),  # Leaky ReLU activation
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),  # Leaky ReLU activation
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))